In [ ]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.7/520.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.8/528.8 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 14.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

Enter your token (input will not be visible): ··········
Add token as git credential? [y/N]: y


In [ ]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"

In [ ]:
huggingface_dataset_name = "tatsu-lab/alpaca"
dataset = load_dataset(huggingface_dataset_name)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
dataset['train'][0]

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [ ]:
model_name='google/gemma-2-2b'
device_map = {"": 0}
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True)

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/288 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
print(f"Tokenizer pad_token (before assignment): {tokenizer.pad_token}")
print(f"Tokenizer pad_token_id (before assignment): {tokenizer.pad_token_id}")

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Tokenizer pad_token (before assignment): <pad>
Tokenizer pad_token_id (before assignment): 0


In [ ]:
# tokenizer.pad_token = tokenizer.eos_token
# print(f"Tokenizer pad_token (after assignment): {tokenizer.pad_token}")
# print(f"Tokenizer pad_token_id (after assignment): {tokenizer.pad_token_id}")
# print(f"Tokenizer pad_token: {tokenizer.pad_token}")
# print(f"Tokenizer pad_token_id: {tokenizer.pad_token_id}")

In [ ]:
def gen(model,formatted_prompt, maxlen=250, sample=True):
  toks = tokenizer(formatted_prompt, return_tensors="pt")
  # print(toks)
  res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.1,num_beams=1,top_p=0.95,).to('cpu')
  return tokenizer.batch_decode(res,skip_special_tokens=True)

In [ ]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 10

# Alpaca fields
instruction = dataset['train'][index]['instruction']
input_text = dataset['train'][index]['input']        # may be empty string
expected_output = dataset['train'][index]['output']

# Alpaca prompt format (same as the 'text' field in dataset)
if input_text:
    formatted_prompt = f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"
else:
    formatted_prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n"

res = gen(original_model, formatted_prompt, 100)

# Split on '### Response:\n' to extract only the generated part
output = res[0].split('### Response:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE EXPECTED OUTPUT:\n{expected_output}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
How did Julius Caesar die?

### Response:

---------------------------------------------------------------------------------------------------
BASELINE EXPECTED OUTPUT:
Julius Caesar was assassinated by a group of up to 60 conspirators, led by Gaius Cassius Longinus and Marcus Junius Brutus, in the Senate House on the Ides of March (15 March) of 44 BC.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Julius Caesar was assassinated by a group of senators who were unhappy with his rule. The assassination was carried out by a group of senators who were unhappy with his rule. The assassination was carried out by a group of senators who were unhappy with his rule. The assassin

In [ ]:
# def create_prompt_formats(sample):
#     """
#     Format various fields of the sample ('instruction','output')
#     Then concatenate them using two newline characters
#     :param sample: Sample dictionnary
#     """
#     INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
#     INSTRUCTION_KEY = "### Instruct: Summarize the below conversation."
#     RESPONSE_KEY = "### Output:"
#     END_KEY = "### End"

#     blurb = f"\n{INTRO_BLURB}"
#     instruction = f"{INSTRUCTION_KEY}"
#     input_context = f"{sample['dialogue']}" if sample["dialogue"] else None
#     response = f"{RESPONSE_KEY}\n{sample['summary']}"
#     end = f"{END_KEY}"

#     parts = [part for part in [blurb, instruction, input_context, response, end] if part]

#     formatted_prompt = "\n\n".join(parts)
#     sample["text"] = formatted_prompt

#     return sample

In [ ]:
# from functools import partial

# # SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
# def get_max_length(model):
#     conf = model.config
#     max_length = None
#     for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
#         max_length = getattr(model.config, length_setting, None)
#         if max_length:
#             print(f"Found max lenth: {max_length}")
#             break
#     if not max_length:
#         max_length = 1024
#         print(f"Using default max length: {max_length}")
#     return max_length


# def preprocess_batch(batch, tokenizer, max_length):
#     """
#     Tokenizing a batch
#     """
#     return tokenizer(
#         batch["text"],
#         max_length=max_length,
#         truncation=True,
#     )

# # SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
# def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, dataset):
#     """Format & tokenize it so it is ready for training
#     :param tokenizer (AutoTokenizer): Model Tokenizer
#     :param max_length (int): Maximum number of tokens to emit from tokenizer
#     """

#     # Add prompt to each sample
#     print("Preprocessing dataset...")
#     dataset = dataset.map(create_prompt_formats)#, batched=True)

#     # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
#     _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
#     dataset = dataset.map(
#         _preprocessing_function,
#         batched=True,
#         remove_columns=['instruction', 'input', 'output', 'text'],
#     )

#     # Filter out samples that have input_ids exceeding max_length
#     dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

#     # Shuffle dataset
#     dataset = dataset.shuffle(seed=seed)

#     return dataset


from functools import partial

def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """
    print("Preprocessing dataset...")

    # Directly tokenize — no need to create 'text', it already exists
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['instruction', 'input', 'output', 'text'],  # remove all 4 columns
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)
    return dataset

In [ ]:
## Pre-process dataset
max_length = get_max_length(original_model)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['train'])
# eval_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['validation'])

Found max lenth: 8192
8192
Preprocessing dataset...


Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

Filter:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# 2 - Using the prepare_model_for_kbit_training method from PEFT
# Preparing the Model for QLoRA
original_model = prepare_model_for_kbit_training(original_model)

In [ ]:
# Print all named modules to find target_modules for Gemma
for name, module in original_model.named_modules():
    print(name)

In [ ]:
print(original_model.config.hidden_size)

2304


## LoRA Hyperparameters Explained

### `r` (Rank)
Controls **how many parameters** LoRA adds. Lower = fewer params, Higher = more expressive.

```
Original Weight Matrix (4096 x 4096) = 16M params
LoRA with r=8  → two small matrices (4096x8) + (8x4096) = 65K params  ✅ tiny!
LoRA with r=32 → (4096x32) + (32x4096) = 262K params
```

| r value | Use case |
|---|---|
| `4 - 8` | Simple tasks, very low memory |
| `16` | Balanced — most common |
| `32` | Complex tasks, more capacity |
| `64+` | Rarely needed, expensive |

**→ Start with `r=16`**

---

### `lora_alpha` (Scaling)
Controls **how much the LoRA update affects** the original weights.

The actual scaling = `lora_alpha / r`

```python
# alpha=32, r=16 → scale = 2.0  (strong update)
# alpha=16, r=16 → scale = 1.0  (neutral)
# alpha=8,  r=16 → scale = 0.5  (soft update)
```

**Simple rule → set `alpha = 2 x r`**
```python
r = 16
lora_alpha = 32  # 2x r is safe default
```

---

### `lora_dropout`
Randomly drops LoRA weights during training to **prevent overfitting**

```python
lora_dropout = 0.0   # small dataset, no dropout
lora_dropout = 0.05  # standard safe default
lora_dropout = 0.1   # if overfitting
```

**→ Just use `0.05` always unless you see overfitting**

---

### `bias`
Whether to train bias terms

```python
bias = "none"     # don't train any bias ← recommended
bias = "all"      # train all biases
bias = "lora_only" # only train lora biases
```
**→ Always use `"none"`** for QLoRA fine-tuning

---

### Your Config for Gemma-2 2B

```python
config = LoraConfig(
    r=16,              # good balance
    lora_alpha=32,     # 2x r
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'o_proj',
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)
```

---

### Quick Decision Guide

```
Small dataset (<10K)  → r=8,  alpha=16, dropout=0.0
Medium dataset        → r=16, alpha=32, dropout=0.05  ✅ your case
Large dataset (100K+) → r=32, alpha=64, dropout=0.05
```

Alpaca has 52K samples → **r=16, alpha=32 is perfect for you.**

# Check your model's hidden size
print(original_model.config)

# or directly
print(original_model.config.hidden_size)
```

You'll see something like:
```
hidden_size: 2304   ← Gemma-2 2B
```

So the real matrix math for YOUR model:
```
# 4096 was example for 7B models
# Gemma-2 2B:

q_proj weight = (2304 x 2304) = 5.3M params

LoRA with r=16:
  Matrix A = (2304 x 16)  = 36K
  Matrix B = (16 x 2304)  = 36K
  Total = 72K  << much smaller than 5.3M ✅
```

---

## What is Bias?

Every linear layer has two parts:
```
Output = (Input × Weight) + Bias
                              ↑
                        this is bias
                        one value per neuron
                        shifts the output up or down
```

Visually:
```
Without bias:  output must pass through zero
With bias:     output can shift freely

y = wx          y = wx + b
    |                |
    passes origin    can shift up/down ← more flexible

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

config = LoraConfig(
    r=16, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'o_proj'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

peft_model = get_peft_model(original_model, config)

In [ ]:
# Split dataset → 90% train, 10% eval
split = train_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split['train']
eval_dataset  = split['test']   # ← this becomes your eval

print(f"Train size: {len(train_dataset)}")
print(f"Eval size:  {len(eval_dataset)}")

Train size: 46801
Eval size:  5201


In [ ]:
import time
import transformers
from transformers import TrainingArguments

output_dir = f'./peft-General-instruction-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=1000,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    weight_decay=0.01,
    optim="paged_adamw_8bit",
    gradient_checkpointing=True,
    bf16=True,
    max_grad_norm=0.3,

    logging_steps=25,
    logging_strategy="steps",
    logging_dir="./logs",

    save_strategy="steps",
    save_steps=25,

    # ❌ removed eval_steps
    # ❌ removed eval_strategy
    # ❌ removed do_eval
    # ❌ removed load_best_model_at_end
    # ❌ removed metric_for_best_model

    report_to="none",
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    # ❌ removed eval_dataset
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [30]:
peft_trainer.train()
# peft_trainer.evaluate()


# # Train
# peft_trainer.train()

# # View loss table after training
# import pandas as pd

# log_history = peft_trainer.state.log_history

# # Separate train and eval logs
# train_logs = [(x['step'], x['loss']) for x in log_history if 'loss' in x]
# eval_logs  = [(x['step'], x['eval_loss']) for x in log_history if 'eval_loss' in x]

# # Merge into DataFrame
# train_df = pd.DataFrame(train_logs, columns=['step', 'train_loss'])
# eval_df  = pd.DataFrame(eval_logs,  columns=['step', 'eval_loss'])

# result = pd.merge(train_df, eval_df, on='step')
# print(result.to_string(index=False))

Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.75 GiB. GPU 0 has a total capacity of 14.56 GiB of which 1.62 GiB is free. Including non-PyTorch memory, this process has 12.95 GiB memory in use. Of the allocated memory 11.56 GiB is allocated by PyTorch, and 1.25 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True)
                                                      # use_auth_token=True)

Loading weights:   0%|          | 0/453 [00:00<?, ?it/s]

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [ ]:
from peft import PeftModel
import os

ft_model = PeftModel.from_pretrained(base_model, os.path.join(output_dir, "checkpoint-1000"), torch_dtype=torch.float16, is_trainable=False)

In [ ]:
%%time
from transformers import set_seed
set_seed(seed)

index = 5
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Instruct: Summarize the following conversation.\n{dialogue}\nOutput:\n"

peft_model_res = gen(ft_model,prompt,250,)
peft_model_output = peft_model_res[0].split('Output:\n')[1]
#print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'input_ids': tensor([[50256, 43993,    25,  5060,  3876,  1096,   262,  1708,  5273,    13,
           198,     2, 15439,    16,     2,    25,   921,   821,  3443,   994,
             0,  1867,  1718,   523,   890,    30,   198,     2, 15439,    17,
             2,    25,   314,  1392,  7819,   287,  4979,   757,    13,  1318,
           373,   257,  7818,  4979, 16853,  1474,   262,  1879,  5420,   454,
         16246,    13,   198,     2, 15439,    16,     2,    25,   632,   338,
          1464,  2138, 22791,   276,   866,   612,  1141, 10484,  1711,    13,
          6674,   345,   815,  1949,   284,  1064,   257,  1180,  6339,   284,
           651,  1363,    13,   198,     2, 15439,    17,     2,    25,   314,
           836,   470,   892,   340,   460,   307, 13941,    11,   284,   307,
          5508,    13,   198,     2, 15439,    16,     2,    25,  3737,   340,
           561,   307,  1365,   611,   345,  2067,  2263,  1171,  4839,  1080,
           284,   670,    13,   198,  

In [ ]:
original_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True)

Loading weights:   0%|          | 0/453 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    human_baseline_text_output = human_baseline_summaries[idx]
    prompt = f"Instruct: Summarize the following conversation.\n{dialogue}\nOutput:\n"

    original_model_res = gen(original_model,prompt,100,)
    original_model_text_output = original_model_res[0].split('Output:\n')[1]

    peft_model_res = gen(ft_model,prompt,100,)
    peft_model_output = peft_model_res[0].split('Output:\n')[1]
    print(peft_model_output)
    peft_model_text_output, success, result = peft_model_output.partition('###')

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])
df

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# INSTRUCTION
I'm not sure if you can do it, but can you try to summarize the sentence below?

# INPUT
The new restaurant in town has received rave reviews for its delicious food and excellent service.

# OUTPUT
The new restaurant in town is highly praised for its delicious food and excellent service.

### 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# INSTRUCTION
I'm not sure if you can do it, but try to summarize the following text in one sentence.

## INPUT
The new restaurant in town offers a wide variety of dishes from different cuisines. The menu includes options for vegetarians, vegans, and meat lovers. The restaurant also has a cozy atmosphere and friendly staff.
##OUTPUT
The new restaurant in town offers a diverse menu with options for vegetarians, vegans, and


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# INSTRUCTION
I'm not sure if you can do it, but try to summarize the following text in one sentence.
## INPUT
The new restaurant in town offers a wide variety of international cuisines, including Italian, Chinese, and Mexican.
##OUTPUT
The new restaurant in town serves a diverse range of international cuisines, such as Italian, Chinese, and Mexican.



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# INSTRUCTION
Write a short summary of the main idea of the following paragraph.

"Plants are living organisms that can make their own food through a process called photosynthesis. They use sunlight, water, and carbon dioxide to produce glucose and oxygen. Plants are essential for life on Earth, as they provide food, oxygen, and habitats for many animals. Plants also have many uses for humans, such as medicine, clothing, and fuel."

## INPUT

##


,human_baseline_summaries,original_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,"# INSTRUCTION\nI'm not sure if you can do it, ...","# INSTRUCTION\nI'm not sure if you can do it, ..."
1,In order to prevent employees from wasting tim...,"# INSTRUCTION\nI'm not sure if you can do it, ...","# INSTRUCTION\nI'm not sure if you can do it, ..."
2,Ms. Dawson takes a dictation for #Person1# abo...,"# INSTRUCTION\nI'm sorry to bother you, but co...","# INSTRUCTION\nI'm not sure if you can do it, ..."
3,#Person2# arrives late because of traffic jam....,"# INSTRUCTION\nI'm sorry to bother you, but co...","# INSTRUCTION\nI'm not sure if you can do it, ..."
4,#Person2# decides to follow #Person1#'s sugges...,"# INSTRUCTION\nI'm sorry to bother you, but co...","# INSTRUCTION\nI'm not sure if you can do it, ..."
5,#Person2# complains to #Person1# about the tra...,"User: I'm sorry to bother you, but could you p...","# INSTRUCTION\nI'm not sure if you can do it, ..."
6,#Person1# tells Kate that Masha and Hero get d...,INSTRUCTION: Write a short summary of the main...,"# INSTRUCTION\nI'm not sure if you can do it, ..."
7,#Person1# tells Kate that Masha and Hero are g...,Instruction: Write a short summary of the main...,"# INSTRUCTION\nI'm not sure if you can do it, ..."
8,#Person1# and Kate talk about the divorce betw...,"# INSTRUCTION\nI'm sorry, but could you please...","# INSTRUCTION\nI'm not sure if you can do it, ..."
9,#Person1# and Brian are at the birthday party ...,INSTRUCTION: Write a short summary of the main...,# INSTRUCTION\nWrite a short summary of the ma...


In [ ]:
import evaluate

rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

ORIGINAL MODEL:
{'rouge1': np.float64(0.08884917593450711), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.06648410211823776), 'rougeLsum': np.float64(0.06957099657800075)}
PEFT MODEL:
{'rouge1': np.float64(0.09229982474233889), 'rouge2': np.float64(0.002127659574468085), 'rougeL': np.float64(0.07105406001220353), 'rougeLsum': np.float64(0.07478699240199224)}
Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 0.35%
rouge2: 0.21%
rougeL: 0.46%
rougeLsum: 0.52%


In [ ]:
def generate_and_compare_summaries(dialogue_input, max_length=100):
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Summarize the below conversation."
    RESPONSE_KEY = "### Output:"

    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{dialogue_input}"
    response_start = f"{RESPONSE_KEY}\n"

    formatted_prompt = "\n\n".join([blurb, instruction, input_context, response_start])

    print(dash_line)
    print(f'INPUT PROMPT:\n{formatted_prompt}')
    print(dash_line)

    # Generate summary with the original model
    original_model_res = gen(original_model, formatted_prompt, max_length)
    original_model_output = original_model_res[0].split(RESPONSE_KEY + '\n')[1]
    print(f'ORIGINAL MODEL GENERATION:\n{original_model_output}')
    print(dash_line)

    # Generate summary with the PEFT model
    peft_model_res = gen(ft_model, formatted_prompt, max_length)
    peft_model_output = peft_model_res[0].split(RESPONSE_KEY + '\n')[1]
    peft_model_output_cleaned, _, _ = peft_model_output.partition('###')
    print(f'PEFT MODEL GENERATION:\n{peft_model_output_cleaned}')
    print(dash_line)

# Example usage:
# Enter your custom dialogue below:
my_dialogue_input = "#Person1#: Hello, how are you today?\n#Person2#: I am fine, thank you. And you?\n#Person1#: I am also fine. What are you doing?\n#Person2#: I am reading a book.\n#Person1#: That's nice."

generate_and_compare_summaries(my_dialogue_input, max_length=250)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruct: Summarize the below conversation.

#Person1#: Hello, how are you today?
#Person2#: I am fine, thank you. And you?
#Person1#: I am also fine. What are you doing?
#Person2#: I am reading a book.
#Person1#: That's nice.

### Output:

---------------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ORIGINAL MODEL GENERATION:

# Python Script for 2.2.2 - The if, elif, and else statements

# In this section, we will learn about the if, elif, and else statements in Python.
# These statements are used to control the flow of the program based on certain conditions.

# Let's start with an example:

# Question 1: Write a program that checks if a number is positive, negative, or zero.

num = int(input("Enter a number: "))

if num > 0:
    print("The number is positive.")
elif num == 0:
    print("The number is zero.")
else:
    print("The number is negative.")

# Explanation:
# We first take input from the user and store it in the variable 'num'.
# Then, we use the if statement to check if the number is greater than zero.
# If it is, we print "The number is positive."
# If it is not, we use the elif statement to check if the number is equal to zero.
# If it is, we print "The number is zero."
# If it is
--------------------------------------------------------------------------------------

# Task
Refine the prompt format for summarization by modifying the `create_prompt_formats` function to use a simpler structure: `Instruct: Summarize the following conversation.` followed by the dialogue, and then `Summary:`. Re-preprocess the dataset with this new format, re-train the PEFT model using the updated datasets, and then evaluate and compare the summaries generated by the fine-tuned PEFT model against the original model using the new prompt format to assess improvements. Finally, analyze and summarize the results of this prompt refinement and model retraining.

## Refine Prompt Format

### Subtask:
Modify the `create_prompt_formats` function and the inference prompt in `generate_and_compare_summaries` to use a simpler structure: `Instruct: Summarize the following conversation.` followed by the dialogue, and then `Summary:`.


**Reasoning**:
The subtask requires modifying the `create_prompt_formats` function and the `generate_and_compare_summaries` function to implement a new prompt format. I will provide a code block that redefines both functions with the specified changes.



In [ ]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = ""
    INSTRUCTION_KEY = "Instruct: Summarize the following conversation."
    RESPONSE_KEY = "Summary:"

    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"\n{sample['dialogue']}" if sample["dialogue"] else None
    response = f"{RESPONSE_KEY} {sample['summary']}"

    parts = [part for part in [INTRO_BLURB, instruction, input_context, response] if part]

    formatted_prompt = "\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

def generate_and_compare_summaries(dialogue_input, max_length=100):
    INTRO_BLURB = ""
    INSTRUCTION_KEY = "Instruct: Summarize the following conversation."
    RESPONSE_KEY = "Summary:"

    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"\n{dialogue_input}"
    response_start = f"{RESPONSE_KEY}"

    formatted_prompt = "\n".join([INTRO_BLURB, instruction, input_context, response_start])

    print(dash_line)
    print(f'INPUT PROMPT:\n{formatted_prompt}')
    print(dash_line)

    # Generate summary with the original model
    original_model_res = gen(original_model, formatted_prompt, max_length)
    original_model_output = original_model_res[0].split(RESPONSE_KEY)[1].strip()
    print(f'ORIGINAL MODEL GENERATION:\n{original_model_output}')
    print(dash_line)

    # Generate summary with the PEFT model
    peft_model_res = gen(ft_model, formatted_prompt, max_length)
    peft_model_output = peft_model_res[0].split(RESPONSE_KEY)[1]
    peft_model_output_cleaned, _, _ = peft_model_output.partition('###') # Keep previous partition logic as model might still generate ###
    print(f'PEFT MODEL GENERATION:\n{peft_model_output_cleaned.strip()}')
    print(dash_line)

# Example usage:
# Enter your custom dialogue below:
my_dialogue_input = "#Person1#: Hello, how are you today?\n#Person2#: I am fine, thank you. And you?\n#Person1#: I am also fine. What are you doing?\n#Person2#: I am reading a book.\n#Person1#: That's nice."

generate_and_compare_summaries(my_dialogue_input, max_length=250)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Instruct: Summarize the following conversation.

#Person1#: Hello, how are you today?
#Person2#: I am fine, thank you. And you?
#Person1#: I am also fine. What are you doing?
#Person2#: I am reading a book.
#Person1#: That's nice.
Summary:
---------------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ORIGINAL MODEL GENERATION:
Once upon a time, in a small town called Mathville, there lived three friends named Alice, Bob, and Carol. They were all passionate about mathematics and loved solving complex problems together. One day, they stumbled upon a mysterious book that contained the secrets of logic and reasoning.

Excitedly, Alice opened the book and discovered a chapter on the importance of logical reasoning in mathematics. She read aloud, "Logical reasoning is crucial in mathematics because it helps us make sense of complex problems and arrive at accurate solutions."

Intrigued, Bob asked, "But how does logical reasoning work? Can you give us an example?"

Alice smiled and replied, "Of course! Let's say we have a mathematical equation: 2x + 5 = 15. To solve for x, we need to isolate it on one side of the equation. We can do this by subtracting 5 from both sides, which gives us 2x = 10. Then, we divide both sides by 2, and we find that x = 5."

Carol nodded in understanding and ad

**Reasoning**:
The previous step updated the prompt formatting functions. Now, the dataset needs to be re-preprocessed using the updated `create_prompt_formats` function to apply the new format to the training and evaluation data.



In [ ]:
## Pre-process dataset
max_length = get_max_length(original_model)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['validation'])

Found max lenth: 2048
2048
Preprocessing dataset...


Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1999 [00:00<?, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Filter:   0%|          | 0/499 [00:00<?, ? examples/s]

**Reasoning**:
The datasets have been re-preprocessed with the new prompt format. Now, the PEFT model needs to be re-trained using these updated training and evaluation datasets to incorporate the new prompt structure.



In [ ]:
import transformers
import time

output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=1000,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none"
)

# Re-initialize peft_model if needed or ensure it's still the Lora model
# If peft_model was overwritten, ensure it's loaded again with LoraConfig
# For simplicity, assuming peft_model is still the Lora-wrapped model from previous steps.
original_model.config.use_cache = False # Ensure this is set for retraining
peft_model = get_peft_model(original_model, config) # Re-apply PEFT adapter to original model if it was reset
peft_model.config.use_cache = False # Setting this again for clarity

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

peft_trainer.train()


`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


Step,Training Loss
25,3.455761
50,2.256328


KeyboardInterrupt: 

# Task
Re-train the PEFT model using the newly preprocessed datasets with the refined prompt format, then load the fine-tuned PEFT model from its checkpoint, evaluate and compare the summaries from both the original base model and the newly retrained PEFT model, analyze and summarize the ROUGE scores and generated summaries for improvements, and finally summarize the overall process and outcomes.

## Modify Prompt Formatting Functions

### Subtask:
Redefine the `create_prompt_formats` function and update the `generate_and_compare_summaries` function to reflect the simpler prompt structure: `Instruct: Summarize the following conversation.` followed by the dialogue, and then `Summary:`. This ensures consistency between training and inference.


**Reasoning**:
The subtask requires redefining the `create_prompt_formats` function and updating the `generate_and_compare_summaries` function to implement a new prompt format. This code block will redefine both functions and then demonstrate the new format with an example.



In [ ]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionnary
    """
    INSTRUCTION_KEY = "Instruct: Summarize the following conversation."
    RESPONSE_KEY = "Summary:"

    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"\n{sample['dialogue']}" if sample["dialogue"] else ""
    response = f"{RESPONSE_KEY} {sample['summary']}"

    # The new format: Instruction\nDialogue\nSummary: Response
    formatted_prompt = f"{instruction}{input_context}\n{response}"
    sample["text"] = formatted_prompt

    return sample

def generate_and_compare_summaries(dialogue_input, max_length=100):
    INSTRUCTION_KEY = "Instruct: Summarize the following conversation."
    RESPONSE_KEY = "Summary:"

    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"\n{dialogue_input}"
    response_start = f"{RESPONSE_KEY}"

    # The new format for inference prompt
    formatted_prompt = f"{instruction}{input_context}\n{response_start}"

    print(dash_line)
    print(f'INPUT PROMPT:\n{formatted_prompt}')
    print(dash_line)

    # Generate summary with the original model
    original_model_res = gen(original_model, formatted_prompt, max_length)
    # Split on the RESPONSE_KEY and take the part after it
    original_model_output = original_model_res[0].split(RESPONSE_KEY)[1].strip()
    print(f'ORIGINAL MODEL GENERATION:\n{original_model_output}')
    print(dash_line)

    # Generate summary with the PEFT model
    peft_model_res = gen(ft_model, formatted_prompt, max_length)
    peft_model_output = peft_model_res[0].split(RESPONSE_KEY)[1]
    # Keep previous partition logic as model might still generate ### (from previous training)
    peft_model_output_cleaned, _, _ = peft_model_output.partition('###')
    print(f'PEFT MODEL GENERATION:\n{peft_model_output_cleaned.strip()}')
    print(dash_line)

# Example usage:
# Enter your custom dialogue below:
my_dialogue_input = "#Person1#: Hello, how are you today?\n#Person2#: I am fine, thank you. And you?\n#Person1#: I am also fine. What are you doing?\n#Person2#: I am reading a book.\n#Person1#: That's nice."

generate_and_compare_summaries(my_dialogue_input, max_length=250)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
Caching is incompatible with gradient checkpointing in PhiDecoderLayer. Setting `past_key_values=None`.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Summarize the following conversation.
#Person1#: Hello, how are you today?
#Person2#: I am fine, thank you. And you?
#Person1#: I am also fine. What are you doing?
#Person2#: I am reading a book.
#Person1#: That's nice.
Summary:
---------------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ORIGINAL MODEL GENERATION:
###
---------------------------------------------------------------------------------------------------
PEFT MODEL GENERATION:
Once upon a time, in a small town called Greenville, there lived a young girl named Lily. Lily was a curious and adventurous girl who loved exploring the world around her. One sunny day, she decided to go on a hike to the nearby mountains.

As Lily climbed up the steep trail, she noticed a beautiful flower blooming on the side of the path. She bent down to get a closer look and saw that the flower was covered in tiny insects. Lily was fascinated by the insects and wanted to learn more about them.

She remembered that her friend, Emma, was studying entomology at the local university. Lily decided to give Emma a call and ask for her help. Emma was thrilled to hear from Lily and agreed to meet her at the flower.

When Emma arrived, she carefully examined the insects on the flower. She explained to Lily that they were called bees and they

# Task
Refine the prompt format for summarization by modifying the `create_prompt_formats` function to use a simpler structure: `Instruct: Summarize the following conversation.` followed by the dialogue, and then `Summary:`. Re-preprocess the dataset with this new format, reduce the `max_steps` for training to 200, re-train the PEFT model using the updated datasets, then load the fine-tuned PEFT model from its `checkpoint-200`, evaluate and compare the summaries from both the original base model and the newly retrained PEFT model, analyze and summarize the ROUGE scores and generated summaries for improvements, and finally summarize the overall process and outcomes.

## Modify Prompt Formatting Functions

### Subtask:
Redefine the `create_prompt_formats` function and update the `generate_and_compare_summaries` function to reflect the simpler prompt structure: `Instruct: Summarize the following conversation.` followed by the dialogue, and then `Summary:`. This ensures consistency between training and inference.


**Reasoning**:
The subtask requires redefining the `create_prompt_formats` function and updating the `generate_and_compare_summaries` function to implement a new prompt format. This code block redefines both functions and then demonstrates the new format with an example.



In [ ]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionnary
    """
    INSTRUCTION_KEY = "Instruct: Summarize the following conversation."
    RESPONSE_KEY = "Summary:"

    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"\n{sample['dialogue']}" if sample["dialogue"] else ""
    response = f"{RESPONSE_KEY} {sample['summary']}"

    # The new format: Instruction\nDialogue\nSummary: Response
    formatted_prompt = f"{instruction}{input_context}\n{response}"
    sample["text"] = formatted_prompt

    return sample

def generate_and_compare_summaries(dialogue_input, max_length=100):
    INSTRUCTION_KEY = "Instruct: Summarize the following conversation."
    RESPONSE_KEY = "Summary:"

    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"\n{dialogue_input}"
    response_start = f"{RESPONSE_KEY}"

    # The new format for inference prompt
    formatted_prompt = f"{instruction}{input_context}\n{response_start}"

    print(dash_line)
    print(f'INPUT PROMPT:\n{formatted_prompt}')
    print(dash_line)

    # Generate summary with the original model
    original_model_res = gen(original_model, formatted_prompt, max_length)
    # Split on the RESPONSE_KEY and take the part after it
    original_model_output = original_model_res[0].split(RESPONSE_KEY)[1].strip()
    print(f'ORIGINAL MODEL GENERATION:\n{original_model_output}')
    print(dash_line)

    # Generate summary with the PEFT model
    peft_model_res = gen(ft_model, formatted_prompt, max_length)
    peft_model_output = peft_model_res[0].split(RESPONSE_KEY)[1]
    # Keep previous partition logic as model might still generate ### (from previous training)
    peft_model_output_cleaned, _, _ = peft_model_output.partition('###')
    print(f'PEFT MODEL GENERATION:\n{peft_model_output_cleaned.strip()}')
    print(dash_line)

# Example usage:
# Enter your custom dialogue below:
my_dialogue_input = "#Person1#: Hello, how are you today?\n#Person2#: I am fine, thank you. And you?\n#Person1#: I am also fine. What are you doing?\n#Person2#: I am reading a book.\n#Person1#: That's nice."

generate_and_compare_summaries(my_dialogue_input, max_length=250)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Summarize the following conversation.
#Person1#: Hello, how are you today?
#Person2#: I am fine, thank you. And you?
#Person1#: I am also fine. What are you doing?
#Person2#: I am reading a book.
#Person1#: That's nice.
Summary:
---------------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ORIGINAL MODEL GENERATION:
#     




















#
















































#     







































































































































#
---------------------------------------------------------------------------------------------------
PEFT MODEL GENERATION:
# INSTRUCTION
Write a short summary of the main idea of the following paragraph.
## INPUT
The human brain is composed of billions of neurons that communicate with each other through chemical and electrical signals. These signals form the basis of our thoughts, memories, emotions, and behaviors. The brain is divided into several regions that perform different functions, such as vision, language, movement, and reasoning. The brain also regulates vital functions, such as breathing, heartbeat, and temperature.

##OUTPUT
The paragraph describes the structure and function of the human brain and its different regions.
----------

**Reasoning**:
The previous training of the PEFT model was interrupted, and the `ft_model` loaded was not trained with the new prompt format. The task requires re-training the PEFT model with the refined prompt format and a reduced `max_steps`. This step will define new training arguments with `max_steps=200`, re-initialize the PEFT model to ensure a clean training run, and then initiate the training process.



In [ ]:
import transformers
import time

output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=200, # Reduced max_steps as per the task
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none"
)

# Re-initialize peft_model to ensure it's a fresh instance with the correct LoraConfig
# This is crucial because the previous training was interrupted, and we want to start clean.
# Re-loading the original_model and applying PEFT config again ensures this.

# First, re-load the base model to ensure it's not a partially trained PEFT model
original_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True)

# Prepare the reloaded original_model for kbit training
original_model = prepare_model_for_kbit_training(original_model)

# Re-apply LoraConfig to get a new peft_model instance for retraining
config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)
peft_model = get_peft_model(original_model, config)

peft_model.config.use_cache = False # Ensure this is set for retraining

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

peft_trainer.train()

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


Loading weights:   0%|          | 0/453 [00:00<?, ?it/s]

Step,Training Loss
25,1.896007
50,1.558537
75,1.565433
100,1.520976
125,1.515898
150,1.555059
175,1.534058
200,1.470140


TrainOutput(global_step=200, training_loss=1.5770134544372558, metrics={'train_runtime': 608.0265, 'train_samples_per_second': 1.316, 'train_steps_per_second': 0.329, 'total_flos': 3372260413685760.0, 'train_loss': 1.5770134544372558, 'epoch': 0.400200100050025})

**Reasoning**:
The PEFT model has been re-trained. Now, I need to load the fine-tuned PEFT model from the latest checkpoint, generate summaries for the test dataset using both the original model and the newly fine-tuned PEFT model, and then calculate and compare their ROUGE scores to evaluate the improvements.



In [ ]:
from peft import PeftModel
import os

# Dynamically get the latest checkpoint from the output_dir
latest_checkpoint = os.path.join(output_dir, 'checkpoint-200') # Assuming max_steps was 200, so the final checkpoint will be checkpoint-200

ft_model = PeftModel.from_pretrained(original_model, latest_checkpoint, torch_dtype=torch.float16, is_trainable=False)

# Create lists to store summaries
original_model_summaries_new = []
peft_model_summaries_new = []

# Ensure the tokenizer is configured for inference
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

# Generate and store summaries for the test set
for idx, dialogue in enumerate(dataset['test'][0:10]['dialogue']):
    prompt = f"Instruct: Summarize the following conversation.\n{dialogue}\nSummary:"

    # Original model summary generation
    original_model_res = gen(original_model, prompt, maxlen=100)
    # Ensure we only get the summary part, handling potential extra text
    original_model_output = original_model_res[0].split('Summary:')[1].strip()
    original_model_summaries_new.append(original_model_output)

    # PEFT model summary generation
    peft_model_res = gen(ft_model, prompt, maxlen=100)
    # Ensure we only get the summary part, handling potential extra text
    peft_model_output = peft_model_res[0].split('Summary:')[1].strip()
    # Further clean up from potential '###' generated by the model
    peft_model_output_cleaned, _, _ = peft_model_output.partition('###')
    peft_model_summaries_new.append(peft_model_output_cleaned.strip())

# Compute ROUGE scores for the new summaries
rouge = evaluate.load('rouge')

original_model_results_new = rouge.compute(
    predictions=original_model_summaries_new,
    references=human_baseline_summaries[0:len(original_model_summaries_new)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results_new = rouge.compute(
    predictions=peft_model_summaries_new,
    references=human_baseline_summaries[0:len(peft_model_summaries_new)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL (New Prompt Format):')
print(original_model_results_new)
print('PEFT MODEL (Re-trained with New Prompt Format):')
print(peft_model_results_new)

print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL (New Prompt Format)")

improvement_new = (np.array(list(peft_model_results_new.values())) - np.array(list(original_model_results_new.values())))
for key, value in zip(peft_model_results_new.keys(), improvement_new):
    print(f'{key}: {value*100:.2f}%')

# Summarize the results from this iteration
print("\n--- Analysis of Prompt Refinement and Retraining ---")
print("After refining the prompt format to 'Instruct: Summarize the following conversation.\n<dialogue>\nSummary:' and retraining the PEFT model for 200 steps:")
print(f"Original Model ROUGE scores (new format): {original_model_results_new}")
print(f"PEFT Model ROUGE scores (retrained with new format): {peft_model_results_new}")
print("The ROUGE scores indicate the following changes:")
for key, value in zip(peft_model_results_new.keys(), improvement_new):
    print(f'  - {key} improved by: {value*100:.2f}%')

print("This comparison shows the impact of the refined prompt format and retraining on the model's summarization capabilities.")


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `e

**Reasoning**:
The warning indicates that `original_model` might already have PEFT configuration when `PeftModel.from_pretrained` is called. To ensure a clean load of the fine-tuned model, I will explicitly load a fresh base model for the `ft_model` before attaching the PEFT adapter.



In [ ]:
from peft import PeftModel
import os

# Dynamically get the latest checkpoint from the output_dir
latest_checkpoint = os.path.join(output_dir, 'checkpoint-200') # Assuming max_steps was 200, so the final checkpoint will be checkpoint-200

# Load a fresh base model instance to attach the PEFT adapter
# This ensures that the base model does not carry any previous PEFT configurations
base_model_for_ft = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True)

ft_model = PeftModel.from_pretrained(base_model_for_ft, latest_checkpoint, torch_dtype=torch.float16, is_trainable=False)

# Create lists to store summaries
original_model_summaries_new = []
peft_model_summaries_new = []

# Ensure the tokenizer is configured for inference
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

# Generate and store summaries for the test set
for idx, dialogue in enumerate(dataset['test'][0:10]['dialogue']):
    prompt = f"Instruct: Summarize the following conversation.\n{dialogue}\nSummary:"

    # Original model summary generation
    original_model_res = gen(original_model, prompt, maxlen=100)
    # Ensure we only get the summary part, handling potential extra text
    original_model_output = original_model_res[0].split('Summary:')[1].strip()
    original_model_summaries_new.append(original_model_output)

    # PEFT model summary generation
    peft_model_res = gen(ft_model, prompt, maxlen=100)
    # Ensure we only get the summary part, handling potential extra text
    peft_model_output = peft_model_res[0].split('Summary:')[1].strip()
    # Further clean up from potential '###' generated by the model
    peft_model_output_cleaned, _, _ = peft_model_output.partition('###')
    peft_model_summaries_new.append(peft_model_output_cleaned.strip())

# Compute ROUGE scores for the new summaries
rouge = evaluate.load('rouge')

original_model_results_new = rouge.compute(
    predictions=original_model_summaries_new,
    references=human_baseline_summaries[0:len(original_model_summaries_new)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results_new = rouge.compute(
    predictions=peft_model_summaries_new,
    references=human_baseline_summaries[0:len(peft_model_summaries_new)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL (New Prompt Format):')
print(original_model_results_new)
print('PEFT MODEL (Re-trained with New Prompt Format):')
print(peft_model_results_new)

print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL (New Prompt Format)")

improvement_new = (np.array(list(peft_model_results_new.values())) - np.array(list(original_model_results_new.values())))
for key, value in zip(peft_model_results_new.keys(), improvement_new):
    print(f'{key}: {value*100:.2f}%')

# Summarize the results from this iteration
print("\n--- Analysis of Prompt Refinement and Retraining ---")
print("After refining the prompt format to 'Instruct: Summarize the following conversation.\n<dialogue>\nSummary:' and retraining the PEFT model for 200 steps:")
print(f"Original Model ROUGE scores (new format): {original_model_results_new}")
print(f"PEFT Model ROUGE scores (retrained with new format): {peft_model_results_new}")
print("The ROUGE scores indicate the following changes:")
for key, value in zip(peft_model_results_new.keys(), improvement_new):
    print(f'  - {key} improved by: {value*100:.2f}%')

print("This comparison shows the impact of the refined prompt format and retraining on the model's summarization capabilities.")

NameError: name 'output_dir' is not defined